In [1]:
import json #to parse json file
import os #to walk through directories
import fnmatch #to match file names
import shutil #to copy data directories

In [2]:
def poly2rect(polygon):
    xs = []
    ys = []
    for i in range(len(polygon)):
        xs.append(float(polygon[i][0]))
        ys.append(float(polygon[i][1]))
    rect = [[min(xs),min(ys)],[max(xs),max(ys)]]
    return rect

def convert2yoloformat(size, rect):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (rect[0][0] + rect[1][0])/2.0
    y = (rect[0][1] + rect[1][1])/2.0
    w = rect[1][0] - rect[0][0]
    h = rect[1][1] - rect[0][1]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [3]:
train_src = '/home/prateek/work/tsd/indian_tsd/' 
train_dest = '/home/prateek/work/tsd/training_set/'
if not os.path.exists(train_dest):
    os.makedirs(train_dest)
train_id = 0


In [4]:
   
for root, dirnames, filenames in os.walk(train_src):
    for filename in fnmatch.filter(filenames, '*.png'):
        json_folder_name = root.split('leftImg8bit/train/')[1]
        json_file_number = filename.split('_leftImg8bit')[0]
        json_file_name = json_file_number + '_gtFine_polygons.json'
        json_path = root.split('leftImg8bit/train/')[0] + 'gtFine/train/'+ json_folder_name + '/' + json_file_name
        #shutil.copy(os.path.join(root,filename),train_dest)
        yolo_label_txt_path = train_dest + filename.replace('png','txt')
        labelFile = open(yolo_label_txt_path,'w+')
        try:
            jsonFile = open(json_path, "r")
            json_data = json.load(jsonFile)
            jsonFile.close()
            objects = json_data["objects"]
            image_width = json_data["imgWidth"]
            image_height = json_data["imgHeight"]
            image_size = [image_width,image_height]
            for index,obj in enumerate(objects):
                if obj["label"] == "traffic sign":
                    poly = json_data["objects"][index]["polygon"]
                    rect = poly2rect(poly)
                    yolo_data = convert2yoloformat(image_size,rect)
                    labelFile.write(str(train_id)+ " "+ " ".join(str(a) for a in yolo_data) +"\n")
        except:
            continue
        labelFile.close()        
